# 🔗 BÀI 5: SQL + PYTHON INTEGRATION

## Mục tiêu:
- Execute SQL from Python
- Parameterized queries
- Batch operations
- Transaction management
- Performance optimization
- Real-world ETL pipeline

In [1]:
# Setup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sys
sys.path.append('/home/jovyan/week-03-04-python-etl/scripts')

from db_connector import DatabaseConnector
from data_cleaner import DataCleaner
from validators import DataValidator
from etl_pipeline import ETLPipeline

# Initialize database connection
db = DatabaseConnector()

print("✅ Setup complete!")
print(f"Database: {db.config['database']}")

2025-12-17 17:38:20,628 - db_connector - INFO - Database connector initialized for data_engineer@postgres


✅ Setup complete!
Database: data_engineer


---
## 📊 PART 1: Basic SQL Execution

In [2]:
# TODO: Execute simple SELECT query
query = """
    SELECT 
        country,
        COUNT(*) as customer_count,
        COUNT(DISTINCT customer_segment) as segments
    FROM analytics.customers
    GROUP BY country
    ORDER BY customer_count DESC
    LIMIT 10
"""

# YOUR CODE HERE
result = db.read_sql(query)

print(f"Query returned {len(result)} rows")
result

/home/jovyan/week-03-04-python-etl/scripts/db_connector.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=params)
2025-12-17 17:38:26,728 - db_connector - INFO - Query executed, DataFrame shape: (10, 3)


Query returned 10 rows


,country,customer_count,segments
0,Ghana,11,3
1,Uruguay,10,3
2,Bahamas,9,3
3,Romania,8,3
4,Portugal,8,3
5,Northern Mariana Islands,8,3
6,Australia,8,3
7,Italy,8,3
8,New Caledonia,8,3
9,Pitcairn Islands,8,3


In [3]:
# TODO: Execute query with date filter
query = """
    SELECT 
        DATE(order_date) as date,
        COUNT(*) as orders,
        SUM(total_amount) as revenue,
        AVG(total_amount) as avg_order_value
    FROM analytics.orders
    WHERE order_date >= CURRENT_DATE - INTERVAL '30 days'
    GROUP BY DATE(order_date)
    ORDER BY date DESC
"""

# YOUR CODE HERE
daily_stats = db.read_sql(query)

print(f"Last 30 days statistics:")
daily_stats.head(10)

/home/jovyan/week-03-04-python-etl/scripts/db_connector.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=params)
2025-12-17 17:38:27,640 - db_connector - INFO - Query executed, DataFrame shape: (27, 4)


Last 30 days statistics:


,date,orders,revenue,avg_order_value
0,2025-12-13,18,85558.56,4753.253333
1,2025-12-12,10,40301.36,4030.136000
2,2025-12-11,14,51609.93,3686.423571
3,2025-12-10,10,40570.81,4057.081000
4,2025-12-09,18,72533.67,4029.648333
5,2025-12-08,9,46702.58,5189.175556
6,2025-12-07,12,53885.99,4490.499167
7,2025-12-06,19,62177.78,3272.514737
8,2025-12-05,15,73639.38,4909.292000
9,2025-12-04,21,79963.36,3807.779048


---
## 🎯 EXERCISE 1: Parameterized Queries

In [ ]:
# TODO: Query with parameters (safe from SQL injection)
def get_customers_by_country(country_name):
    """
    Get customers from specific country using parameterized query
    """
    query = """
        SELECT *
        FROM analytics.customers
        WHERE country = %s
        LIMIT 100
    """
    
    # YOUR CODE HERE
    return db.read_sql(query, (country_name,))

# Test
vietnam_customers = get_customers_by_country('Vietnam')
print(f"Vietnam customers: {len(vietnam_customers)}")
vietnam_customers.head()

In [ ]:
# TODO: Query with multiple parameters
def get_orders_by_date_range(start_date, end_date, min_amount=0):
    """
    Get orders within date range and minimum amount
    """
    query = """
        SELECT *
        FROM analytics.orders
        WHERE order_date BETWEEN %s AND %s
          AND total_amount >= %s
        ORDER BY order_date DESC
    """
    
    # YOUR CODE HERE
    return db.read_sql(query, (start_date, end_date, min_amount))

# Test
start = datetime.now() - timedelta(days=30)
end = datetime.now()
orders = get_orders_by_date_range(start, end, min_amount=1000)

print(f"Found {len(orders)} orders")
orders.head()

---
## 🎯 EXERCISE 2: Write Operations

In [ ]:
# TODO: Create a summary table
create_table_sql = """
    DROP TABLE IF EXISTS analytics.daily_summary;
    
    CREATE TABLE analytics.daily_summary (
        summary_date DATE PRIMARY KEY,
        total_orders INTEGER,
        total_revenue DECIMAL(15,2),
        avg_order_value DECIMAL(10,2),
        unique_customers INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
"""

# YOUR CODE HERE
db.execute_query(create_table_sql, fetch=False)

print("✅ Table created successfully")

In [ ]:
# TODO: Calculate and insert daily summary
calculate_summary_sql = """
    INSERT INTO analytics.daily_summary 
    (summary_date, total_orders, total_revenue, avg_order_value, unique_customers)
    SELECT 
        DATE(order_date) as summary_date,
        COUNT(*) as total_orders,
        SUM(total_amount) as total_revenue,
        AVG(total_amount) as avg_order_value,
        COUNT(DISTINCT customer_id) as unique_customers
    FROM analytics.orders
    WHERE order_date >= CURRENT_DATE - INTERVAL '30 days'
    GROUP BY DATE(order_date)
    ON CONFLICT (summary_date) 
    DO UPDATE SET
        total_orders = EXCLUDED.total_orders,
        total_revenue = EXCLUDED.total_revenue,
        avg_order_value = EXCLUDED.avg_order_value,
        unique_customers = EXCLUDED.unique_customers,
        created_at = CURRENT_TIMESTAMP;
"""

# YOUR CODE HERE
db.execute_query(calculate_summary_sql, fetch=False)

print("✅ Summary calculated and inserted")

# Verify
summary = db.read_sql("SELECT * FROM analytics.daily_summary ORDER BY summary_date DESC LIMIT 10")
summary

---
## 🎯 EXERCISE 3: Batch Operations

In [ ]:
# TODO: Write DataFrame to database in batches
# Create sample data
sample_data = pd.DataFrame({
    'product_id': range(1, 101),
    'product_name': [f'Product {i}' for i in range(1, 101)],
    'category': np.random.choice(['Electronics', 'Clothing', 'Food', 'Books'], 100),
    'price': np.random.uniform(10, 1000, 100).round(2),
    'stock': np.random.randint(0, 100, 100)
})

print(f"Sample data shape: {sample_data.shape}")
sample_data.head()

In [ ]:
# TODO: Write to database
# YOUR CODE HERE
rows_written = db.write_dataframe(
    sample_data,
    table_name='products_temp',
    schema='analytics',
    if_exists='replace'
)

print(f"✅ Written {rows_written} rows to database")

# Verify
verify = db.read_sql("SELECT * FROM analytics.products_temp LIMIT 10")
verify

---
## 🎯 EXERCISE 4: Transaction Management

In [ ]:
# TODO: Execute multiple queries in a transaction
def update_customer_segment():
    """
    Update customer segments based on order history
    Uses transaction to ensure all-or-nothing
    """
    queries = [
        # Reset all to 'Standard'
        """
        UPDATE analytics.customers
        SET customer_segment = 'Standard'
        """,
        
        # Update to Premium (>10 orders or >$5000 total)
        """
        UPDATE analytics.customers c
        SET customer_segment = 'Premium'
        FROM (
            SELECT customer_id
            FROM analytics.orders
            GROUP BY customer_id
            HAVING COUNT(*) > 10 OR SUM(total_amount) > 5000
        ) o
        WHERE c.customer_id = o.customer_id
        """,
        
        # Update to VIP (>20 orders or >$10000 total)
        """
        UPDATE analytics.customers c
        SET customer_segment = 'VIP'
        FROM (
            SELECT customer_id
            FROM analytics.orders
            GROUP BY customer_id
            HAVING COUNT(*) > 20 OR SUM(total_amount) > 10000
        ) o
        WHERE c.customer_id = o.customer_id
        """
    ]
    
    # YOUR CODE HERE
    conn = db.get_connection()
    cursor = conn.cursor()
    
    try:
        for query in queries:
            cursor.execute(query)
        
        conn.commit()
        print("✅ Transaction committed successfully")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Transaction rolled back: {e}")
        raise
    
    finally:
        cursor.close()
        conn.close()

# Test (commented out to avoid changing data)
# update_customer_segment()

# Check current distribution
segment_dist = db.read_sql("""
    SELECT customer_segment, COUNT(*) as count
    FROM analytics.customers
    GROUP BY customer_segment
    ORDER BY count DESC
""")

print("Current segment distribution:")
segment_dist

---
## 🎯 EXERCISE 5: Complete ETL Pipeline

In [ ]:
# TODO: Build customer analytics pipeline
def customer_analytics_pipeline():
    """
    Complete ETL pipeline:
    1. Extract customers and orders
    2. Calculate customer metrics
    3. Clean and validate
    4. Load to analytics table
    """
    pipeline = ETLPipeline('customer_analytics')
    
    try:
        # EXTRACT
        print("\n" + "="*60)
        print("EXTRACT PHASE")
        print("="*60)
        
        customers = db.read_sql("SELECT * FROM analytics.customers")
        print(f"✅ Extracted {len(customers)} customers")
        
        orders = db.read_sql("SELECT * FROM analytics.orders")
        print(f"✅ Extracted {len(orders)} orders")
        
        # TRANSFORM
        print("\n" + "="*60)
        print("TRANSFORM PHASE")
        print("="*60)
        
        # Calculate metrics
        customer_metrics = orders.groupby('customer_id').agg({
            'order_id': 'count',
            'total_amount': ['sum', 'mean', 'max'],
            'order_date': ['min', 'max']
        }).reset_index()
        
        customer_metrics.columns = [
            'customer_id', 'total_orders', 'total_spent',
            'avg_order_value', 'max_order_value',
            'first_order_date', 'last_order_date'
        ]
        
        print(f"✅ Calculated metrics for {len(customer_metrics)} customers")
        
        # Merge with customer data
        analytics_df = customers.merge(
            customer_metrics,
            on='customer_id',
            how='left'
        )
        
        # Fill nulls for customers without orders
        analytics_df['total_orders'] = analytics_df['total_orders'].fillna(0)
        analytics_df['total_spent'] = analytics_df['total_spent'].fillna(0)
        
        # Add derived columns
        analytics_df['days_since_last_order'] = (
            pd.Timestamp.now() - pd.to_datetime(analytics_df['last_order_date'])
        ).dt.days
        
        analytics_df['customer_lifetime_days'] = (
            pd.to_datetime(analytics_df['last_order_date']) - 
            pd.to_datetime(analytics_df['first_order_date'])
        ).dt.days
        
        print(f"✅ Created analytics dataset: {analytics_df.shape}")
        
        # VALIDATE
        print("\n" + "="*60)
        print("VALIDATION PHASE")
        print("="*60)
        
        validator = (
            DataValidator(analytics_df, "customer_analytics")
            .check_not_null(['customer_id', 'customer_name'])
            .check_unique(['customer_id'])
            .check_range('total_orders', 0, float('inf'))
            .check_range('total_spent', 0, float('inf'))
        )
        
        validator.print_report()
        
        # LOAD
        print("\n" + "="*60)
        print("LOAD PHASE")
        print("="*60)
        
        rows = db.write_dataframe(
            analytics_df,
            table_name='customer_analytics',
            schema='analytics',
            if_exists='replace'
        )
        
        print(f"✅ Loaded {rows} rows to analytics.customer_analytics")
        
        print("\n" + "="*60)
        print("PIPELINE COMPLETED SUCCESSFULLY")
        print("="*60)
        
        return analytics_df
        
    except Exception as e:
        print(f"\n❌ Pipeline failed: {e}")
        raise

# Run pipeline
result = customer_analytics_pipeline()

print("\n📊 Sample results:")
result.head(10)

In [ ]:
# Verify loaded data
verify_query = """
    SELECT 
        customer_segment,
        COUNT(*) as customers,
        AVG(total_orders) as avg_orders,
        AVG(total_spent) as avg_spent,
        AVG(days_since_last_order) as avg_days_since_order
    FROM analytics.customer_analytics
    GROUP BY customer_segment
    ORDER BY avg_spent DESC
"""

segment_analysis = db.read_sql(verify_query)
print("\n📈 Customer Segment Analysis:")
segment_analysis

---
## 🎯 CHALLENGE: Build Your Own Pipeline

In [ ]:
# TODO: Create product performance pipeline
# Requirements:
# 1. Extract orders and order_items
# 2. Calculate per product:
#    - Total quantity sold
#    - Total revenue
#    - Number of orders
#    - Average price
#    - First/last sale date
# 3. Validate results
# 4. Load to analytics.product_performance

def product_performance_pipeline():
    """
    YOUR CODE HERE
    """
    pass

# Run your pipeline
# product_performance_pipeline()

---
## 📚 KEY TAKEAWAYS

### SQL + Python Integration Best Practices:

1. **Use Parameterized Queries** - Prevent SQL injection
2. **Batch Operations** - Process large datasets efficiently
3. **Transaction Management** - Ensure data consistency
4. **Error Handling** - Rollback on failures
5. **Connection Pooling** - Reuse connections
6. **Validate Data** - Check before and after loading
7. **Log Everything** - Track pipeline execution
8. **Optimize Queries** - Use indexes, limit data transfer

### When to Use SQL vs Python:

**Use SQL when:**
- Filtering large datasets
- Joining tables
- Aggregating data
- Working with database-specific features

**Use Python when:**
- Complex transformations
- Machine learning
- API integrations
- Custom business logic

**Best approach:** Combine both!
- SQL for heavy lifting in database
- Python for complex logic and orchestration

---
## 🎓 CONGRATULATIONS!

Bạn đã hoàn thành Week 3-4: Python for Data Engineering!

### Bạn đã học được:
✅ Pandas for data manipulation
✅ Data cleaning techniques
✅ ETL patterns and best practices
✅ Data validation frameworks
✅ SQL + Python integration

### Tiếp theo:
- Week 5-6: Apache Airflow & Workflow Orchestration
- Week 7-8: Data Warehousing with dbt
- Week 9-10: Real-time Data Processing